# Predicting Heart Disease Using Machine Learning

This notebook looks into using various Python-based machine learning and data science libraries in an attempt to build a machine learning model capabale of predicting whether or not someone has heart disease based on their mendical attributes.